<a href="https://colab.research.google.com/github/LuccasTraumer/ibjjf-data/blob/main/Brasileiro_IBBJF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import requests
import json
import re

# URL base dos JSONs
base_url = "https://raw.githubusercontent.com/LuccasTraumer/ibjjf-data/main/brasileiro-ibjjf/"
anos = list(range(1994, 2024))

# Mapeamento para normalizar faixas
mapa_faixas = {
    "white": "WHITE", "white belt": "WHITE",
    "blue": "BLUE", "blue belt": "BLUE",
    "purple": "PURPLE", "purple belt": "PURPLE",
    "brown": "BROWN", "brown belt": "BROWN",
    "black": "BLACK", "black belt": "BLACK",
    "coral": "CORAL", "coral belt": "CORAL",
    "red": "RED", "red belt": "RED"
}

# Mapeamento para normalizar sexo
mapa_sexo = {
    "male": "MALE", "m": "MALE", "masculino": "MALE",
    "female": "FEMALE", "f": "FEMALE", "feminino": "FEMALE"
}

dfs = []

for ano in anos:
    url = f"{base_url}{ano}.json"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = json.loads(response.text)

        linhas = []
        for item in data:
            categoria_raw = item.get("categoria", "").strip()
            ranking = item.get("ranking", [])

            partes = [p.strip() for p in categoria_raw.split(" / ")]
            divisao = partes[0] if len(partes) > 0 else None
            faixa_raw = partes[1] if len(partes) > 1 else None
            sexo_raw = partes[2] if len(partes) > 2 else None
            categoria_peso = partes[3] if len(partes) > 3 else None

            # Normaliza faixa e sexo
            faixa = None
            sexo = None
            if faixa_raw:
                faixa_lower = faixa_raw.lower().replace("belt", "").strip()
                faixa = mapa_faixas.get(faixa_lower, faixa_raw.upper())
            if sexo_raw:
                sexo_lower = sexo_raw.lower().strip()
                sexo = mapa_sexo.get(sexo_lower, sexo_raw.upper())

            for posicao, atleta in enumerate(ranking, start=1):
                linhas.append({
                    "ano": ano,
                    "competicao": "Brasileiro IBJJF",
                    "divisao": divisao,
                    "faixa": faixa,
                    "sexo": sexo,
                    "categoria_peso": categoria_peso,
                    "posicao": posicao,
                    "nome": atleta.get("nome"),
                    "academia": atleta.get("academia")
                })

        df = pd.DataFrame(linhas)
        dfs.append(df)
        print(f"✅ {ano} carregado — {len(df)} atletas")

    except Exception as e:
        print(f"⚠️ Erro ao carregar {ano}: {e}")

dados_bjj = pd.concat(dfs, ignore_index=True)
print(f"\nTotal de registros: {len(dados_bjj)}")

# Preview dos dados
dados_bjj.head(10)


✅ 1994 carregado — 76 atletas
⚠️ Erro ao carregar 1995: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/LuccasTraumer/ibjjf-data/main/brasileiro-ibjjf/1995.json
✅ 1996 carregado — 404 atletas
✅ 1997 carregado — 438 atletas
✅ 1998 carregado — 778 atletas
✅ 1999 carregado — 984 atletas
✅ 2000 carregado — 735 atletas
✅ 2001 carregado — 853 atletas
✅ 2002 carregado — 910 atletas
✅ 2003 carregado — 928 atletas
✅ 2004 carregado — 579 atletas
✅ 2005 carregado — 796 atletas
✅ 2006 carregado — 711 atletas
✅ 2007 carregado — 808 atletas
✅ 2008 carregado — 751 atletas
✅ 2009 carregado — 844 atletas
✅ 2010 carregado — 952 atletas
✅ 2011 carregado — 1131 atletas
✅ 2012 carregado — 1119 atletas
✅ 2013 carregado — 1117 atletas
✅ 2014 carregado — 1273 atletas
✅ 2015 carregado — 1582 atletas
✅ 2016 carregado — 1858 atletas
✅ 2017 carregado — 2173 atletas
✅ 2018 carregado — 2579 atletas
✅ 2019 carregado — 2781 atletas
⚠️ Erro ao carregar 2020: 404 Client Error: Not Found for url: 

,ano,competicao,divisao,faixa,sexo,categoria_peso,posicao,nome,academia
0,1994,Brasileiro IBJJF,Adult,BLUE,MALE,Rooster,1,Robson Medeiros,Pitbull
1,1994,Brasileiro IBJJF,Adult,BLUE,MALE,Rooster,2,Bruno Ximenes,Sergio Souza
2,1994,Brasileiro IBJJF,Adult,BLUE,MALE,Light Feather,1,Thiago Asfora,Nova Geracao
3,1994,Brasileiro IBJJF,Adult,BLUE,MALE,Light Feather,2,Fredson Alves,Fed. Amazonas
4,1994,Brasileiro IBJJF,Adult,BLUE,MALE,Feather,1,Leonardo Borges,Carlson Gracie
5,1994,Brasileiro IBJJF,Adult,BLUE,MALE,Feather,2,Gustavo Miggiati,Gracie Barra
6,1994,Brasileiro IBJJF,Adult,BLUE,MALE,Light,1,Rafael Correa,Gracie Barra
7,1994,Brasileiro IBJJF,Adult,BLUE,MALE,Light,2,Rogerio Miranda,Carlson Gracie
8,1994,Brasileiro IBJJF,Adult,BLUE,MALE,Middle,1,Roberto Atalla,Gracie Barra
9,1994,Brasileiro IBJJF,Adult,BLUE,MALE,Middle,2,Eduardo Lerner,Carlson Gracie


In [3]:
academias = (
    dados_bjj[dados_bjj["posicao"] == 1]
    .groupby(["sexo", "academia"])
    .size()
    .reset_index(name="titulos")
    .sort_values(["sexo", "titulos"], ascending=[True, False])
)
academias.head(20)


,sexo,academia,titulos
14,AZUL,Atos Jiu-Jitsu,23
56,AZUL,Gracie Barra,18
72,AZUL,Nova União,17
5,AZUL,Alliance,14
51,AZUL,GF Team,14
24,AZUL,CheckMat,9
79,AZUL,Qatar BJJ / Vision Brasil,9
36,AZUL,Double Five,7
37,AZUL,Dream Art,5
39,AZUL,DreamArt,5


In [4]:
# Atletas mais recorrentes
atletas = (
    dados_bjj.groupby("nome")
    .size()
    .sort_values(ascending=False)
)
atletas.head(10)


,0
nome,
Carlos Eduardo,23
Carlos Alberto,20
Schumann de Lyra Carvalho,19
Marco Antonio,19
Maria do Carmo Paixão Teixeira,18
Leticia Seguetto Tanabe Lalli,18
Paulo Cesar,18
Willian Barbosa Siqueira,17
Célia Regina Honorato de Oliveira,17


In [5]:
# Evolução da quantidade de participantes por ano
import plotly.express as px

contagem_ano = (
    dados_bjj.groupby("ano")
    .size()
    .reset_index(name="quantidade")
)

px.bar(contagem_ano, x="ano", y="quantidade", title="Número de atletas registrados por ano")


In [6]:
# Total de participações por atleta (independente da colocação)
participacoes = (
    dados_bjj
    .groupby(["sexo", "nome"])
    .size()
    .reset_index(name="participacoes")
    .sort_values(["sexo", "participacoes"], ascending=[True, False])
)

# Top 10 masculinos e femininos
top_masculino = participacoes[participacoes["sexo"] == "MALE"].head(10)
top_feminino  = participacoes[participacoes["sexo"] == "FEMALE"].head(10)

print("🏋️‍♂️ Top 10 Atletas Masculinos que mais competiram:")
display(top_masculino)

print("🏋️‍♀️ Top 10 Atletas Femininos que mais competiram:")
display(top_feminino)


🏋️‍♂️ Top 10 Atletas Masculinos que mais competiram:


,sexo,nome,participacoes
10269,MALE,Carlos Eduardo,23
10234,MALE,Carlos Alberto,20
13905,MALE,Marco Antonio,19
14666,MALE,Paulo Cesar,18
9215,MALE,Admilson Brites,16
12740,MALE,Jorge Henrique,14
12927,MALE,Julio Cesar,14
9741,MALE,Antonio Carlos,13
13922,MALE,Marco Aurelio,13
14818,MALE,Pedro Henrique,13


🏋️‍♀️ Top 10 Atletas Femininos que mais competiram:


,sexo,nome,participacoes
6961,FEMALE,Mariana Coelho,13
6225,FEMALE,Bianca Andrade,12
7275,FEMALE,Silvana Abreu,11
6496,FEMALE,Fabiana Borges,10
7032,FEMALE,Mirela Cortes,10
6567,FEMALE,Gilda Maria,8
6956,FEMALE,Maria do Carmo,8
6523,FEMALE,Fernanda Mazelli,7
6130,FEMALE,Ana Carolina,6
6165,FEMALE,Andrea Endy,6


In [7]:
# Apenas os campeões (posição = 1)
campeoes = (
    dados_bjj[dados_bjj["posicao"] == 1]
    .groupby(["sexo", "nome"])
    .size()
    .reset_index(name="titulos")
    .sort_values(["sexo", "titulos"], ascending=[True, False])
)

# Top 10 de cada sexo
top_campeoes_masc = campeoes[campeoes["sexo"] == "MALE"].head(10)
top_campeoes_fem  = campeoes[campeoes["sexo"] == "FEMALE"].head(10)

print("🥇 Top 10 Campeões (Masculino):")
display(top_campeoes_masc)

print("🥇 Top 10 Campeãs (Feminino):")
display(top_campeoes_fem)


🥇 Top 10 Campeões (Masculino):


,sexo,nome,titulos
3603,MALE,Admilson Brites,12
4001,MALE,Carlos Alberto,10
5617,MALE,Mauricio Robbe,10
4567,MALE,Fredson Paixão,9
5172,MALE,Leonardo Correa,9
3816,MALE,Antonio Carlos,8
5808,MALE,Paulo Cesar,8
5004,MALE,Jorge Henrique,7
5911,MALE,Philipe Freitas,7
3928,MALE,Bluno Rafael,6


🥇 Top 10 Campeãs (Feminino):


,sexo,nome,titulos
2602,FEMALE,Mirela Cortes,9
2562,FEMALE,Maria do Carmo,8
2116,FEMALE,Bianca Andrade,7
2744,FEMALE,Silvana Abreu,7
2266,FEMALE,Erica Paes,6
2275,FEMALE,Fabiana Borges,6
2117,FEMALE,Bianca Andrade Barreto,5
2318,FEMALE,Gilda Maria,5
2380,FEMALE,Janaina Ventura,5
2674,FEMALE,Priscila Cardoso,5


In [8]:
academias = (
    dados_bjj[dados_bjj["posicao"] == 1]
    .groupby(["sexo", "academia"])
    .size()
    .reset_index(name="titulos")
    .sort_values(["sexo", "titulos"], ascending=[True, False])
)
academias.head(10)


,sexo,academia,titulos
14,AZUL,Atos Jiu-Jitsu,23
56,AZUL,Gracie Barra,18
72,AZUL,Nova União,17
5,AZUL,Alliance,14
51,AZUL,GF Team,14
24,AZUL,CheckMat,9
79,AZUL,Qatar BJJ / Vision Brasil,9
36,AZUL,Double Five,7
37,AZUL,Dream Art,5
39,AZUL,DreamArt,5


In [9]:
import plotly.express as px

px.bar(
    top_campeoes_masc,
    x="titulos",
    y="nome",
    orientation="h",
    title="🥇 Top 10 Campeões (Masculino)",
    text="titulos"
)

px.bar(
    top_campeoes_fem,
    x="titulos",
    y="nome",
    orientation="h",
    title="🥇 Top 10 Campeãs (Feminino)",
    text="titulos"
)


In [10]:
# Academias com mais campeões (1º lugar)
top_academias = (
    dados_bjj[dados_bjj["posicao"] == 1]
    .groupby("academia")
    .size()
    .sort_values(ascending=False)
    .head(10)
)
top_academias


,0
academia,
Gracie Barra,753
Alliance,670
Nova União,498
GF Team,478
Gracie Humaita,342
CheckMat,320
UGF,203
Osvaldo Alves,202
Brazilian Top Team,198
